In [7]:
from rdflib.plugins.sparql import parser

In [8]:
class Print_Visitor:
    
    def visit(self, ast, lvl=0):
        if type(ast) == parser.ParseResults:
            self.results(ast, lvl)
            for j in ast.as_list(): self.visit(j, lvl+1)
        
        elif type(ast) == list:
            for j in ast: self.visit(j, lvl+1)
       
        elif hasattr(ast, 'items') and len(ast) > 0:
            self.node(ast, lvl)
            for j in ast.values(): self.visit(j, lvl+1)
        
        else:
            return self.leaf(ast, lvl)
    
    def __indent(self, lvl):
        return " ".join([ "" for _ in range(lvl) ])
    
    def results(self, ast, lvl):
        print(self.__indent(lvl), "results:", ast)
        
    def node(self, ast, lvl):
        print(self.__indent(lvl), "node:", ast.name)
    
    def leaf(self, ast, lvl):
        print(self.__indent(lvl), "leaf:", ast)

In [ ]:
class To_N3_Visitor:
    
    def convert(self, query):
        if 'projection' not in query:
            head = "{ :x :result true }"
        else:
            vars = [ f"('{var.var}' {var.var.n3()})" for var in query['projection'] ]
            vars = "( " + " ".join(vars) + " )"
            head = "{ " + f"_:x :result {vars}" + " }"
        
        body = self.visit(query['where'])
        body = "{ " + body + " }"
        
        return f"{body}\n => {head} ."
    
    def visit(self, ast):
        if type(ast) == parser.ParseResults:
            ret = [ self.visit(j) for j in ast.as_list() ]
            return " . ".join([ " ".join(ret[n*3 : (n+1)*3]) for n in range(int(len(ret)/3)) ])
                    
        elif type(ast) == list:
            return " ".join([ self.visit(j) for j in ast ])
       
        elif hasattr(ast, 'name'): # node
            if ast.name == 'pname':
                return self.leaf(ast)        
            elif ast.name == 'TriplesBlock':
                ret = [ self.visit(j) for j in ast['triples'] ]
                return " . ".join(ret)
            else:
                return self.node(ast)
        
        else:
            return self.leaf(ast)
        
    def node(self, ast):        
        symbol = None; infix = False
        match(ast.name):
            case 'PathAlternative':
                symbol = "|"; infix = True
            case 'PathSequence':
                symbol = "/"; infix = True
            case 'PathNegatedPropertySet':
                symbol = "!"; infix = False
            case 'PathEltOrInverse':
                symbol = "^"; infix = False
                
        # kleene-star: represented using "mod" key
        
        key = list(ast.keys())[0]
        is_list = (type(ast[key]) == list)
        nodes = [ self.visit(j) for j in ast[key] ] if is_list else [ self.visit(ast[key]) ]

        if symbol is not None:
            symbol = f" '{symbol}' "
            sep = symbol if infix else " "
            nodes_str = "(" + sep.join(nodes) + ")" if len(nodes) > 1 else sep.join(nodes)
            if not infix:
                nodes_str = "(" + symbol + nodes_str + ")"
            return nodes_str
        else:
            return nodes[0]
        
    def leaf(self, ast):
        if hasattr(ast, "n3") and ast.n3 is not None:
            return ast.n3()
        elif ast.name == "pname":
            return ":" + ast['localname']
        else:
            return ast.__str__()

In [10]:
query = "SELECT ?x WHERE { ?x (:p1/:p2)* ?z ; !(:p3|:p4|:p5) ?a }"
query = \
"PREFIX : <http://example.org/gmark/> " + \
"SELECT DISTINCT ?x0 ?x3 WHERE {  { " + \
"?x0 (((^:p1/:p1)|(^:p1/:p1/^:p1/:p1))) ?x1 . ?x1 (((^:p1/:p1/^:p1/:p1)|(^:p1/:p1))) ?x2 . ?x2 ((^:p1/:p1)|(^:p1/:p1/^:p1/:p1)) ?x3 . " + \
"} }"

query = parser.parseQuery(query)
query = query[1]

In [11]:
print(To_N3_Visitor().convert(query))

{ ?x0 ((( '^' :p1) '/' :p1) '|' (( '^' :p1) '/' :p1 '/' ( '^' :p1) '/' :p1)) ?x1 . ?x1 ((( '^' :p1) '/' :p1 '/' ( '^' :p1) '/' :p1) '|' (( '^' :p1) '/' :p1)) ?x2 . ?x2 ((( '^' :p1) '/' :p1) '|' (( '^' :p1) '/' :p1 '/' ( '^' :p1) '/' :p1)) ?x3 }
 => { _:x :result ( ('x0' ?x0) ('x3' ?x3) ) } .


In [12]:
import os

visitor = To_N3_Visitor()

path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/gmark_50_mix"
for file in os.listdir(path):
    if not file.endswith(".sparql"):
        continue
    print(file)
    with open(os.path.join(path, file), 'r') as fh:
        query = fh.read()
        query = parser.parseQuery(query)
        query = query[1]
        
        conv = visitor.convert(query)
        conv = "@prefix : <http://example.org/gmark/> .\n\n" + conv
        print(conv)
        
        n3_file = file[0:file.index(".")] + ".n3"
        with open(os.path.join(path, "n3", n3_file), 'w') as fh2:
            fh2.write(conv)
            
        print("\n")

query-24.sparql
@prefix : <http://example.org/gmark/> .

{ ?x0 ((:p0 '/' ( '^' :p0) '/' :p0 '/' ( '^' :p0)) '|' (:p0 '/' :p3 '/' ( '^' :p3) '/' ( '^' :p0)) '|' (:p0 '/' :p3 '/' ( '^' :p3) '/' ( '^' :p0))) ?x1 . ?x1 ((:p0 '/' :p1 '/' ( '^' :p1) '/' :p1) '|' (:p0 '/' :p1 '/' ( '^' :p1) '/' :p1) '|' (:p0 '/' :p1 '/' ( '^' :p1) '/' :p1)) ?x2 . ?x2 ((( '^' :p1) '/' :p1 '/' ( '^' :p1) '/' :p3) '|' (( '^' :p1) '/' :p3) '|' (( '^' :p1) '/' :p3)) ?x3 }
 => { _:x :result ( ('x0' ?x0) ('x1' ?x1) ('x3' ?x3) ('x2' ?x2) ) } .


query-47.sparql
@prefix : <http://example.org/gmark/> .

{ ?x0 (( '^' :p2) '/' :p2 '/' ( '^' :p2) '/' :p2) ?x1 . ?x0 (( '^' :p2) '/' ( '^' :p1) '/' :p3) ?x2 . ?x2 ((( '^' :p3) '/' :p1 '/' :p2) '|' (( '^' :p3) '/' :p1 '/' :p2)) ?x1 }
 => { _:x :result ( ('x0' ?x0) ('x2' ?x2) ('x1' ?x1) ) } .


query-36.sparql
@prefix : <http://example.org/gmark/> .

{ ?x0 ((( '^' :p2) '/' ( '^' :p1) '/' :p1) '|' (( '^' :p2) '/' :p2 '/' ( '^' :p2))) ?x1 . ?x1 ((:p2 '/' ( '^' :p2) '/' ( '^' :p1)